### 예진ver 문장 토큰화

In [4]:
import re
from kiwipiepy import Kiwi
kiwi = Kiwi()
import glob
import os
import pandas as pd

def remove_policy_date(text):
    # "통화정책방향 (YYYY. M. D.)" 또는 "통화정책방향 (YYYY. M. D)" 패턴 제거
    pattern = r"통화정책방향\s*\(\d{4}\.\s*\d{1,2}\.\s*\d{1,2}\.?\)"
    clean_text = re.sub(pattern, "", text).strip()
    return clean_text
data_folder = r'C:\Users\yoons\Desktop\금리예측포르젝트\test\병합할파일\기자회견테스트폴더\conferences_sentences.csv'
data_files = glob.glob(os.path.join(data_folder, '*.txt'))

dates = []
sentences = []

for d in data_files:
    f = open(d, encoding='utf-8')
    data = f.read()

    # 각 문서에서 날짜 뽑아내기 (형식 일정)
    date_match = re.findall(r'(\d{4}).\s*(\d{1,2}).\s*(\d{1,2})', data)

    if date_match :
        year, month, day = date_match[0]  # 첫 번째 매치만 사용
        if len(month) == 1 :
            month = '0' + month
        if len(day) == 1 :
            day = '0' + day
        date_f = f"{year}-{month}-{day}"
        print(date_f) # 정상적으로 출력되는 것 확인 가능
    else:
        date_f = "날짜 없음"
    dates.append(date_f)

    data = remove_policy_date(data)
    # 텍스트에서는 특수문자 □ 제거
    data = data.replace('□','')
    data = data.replace('\n', '')

    # 나머지는 Kiwi 이용해서 문장 토큰화시킨 담에 df 저장하기 
    sents = [s.text for s in kiwi.split_into_sents(data)]
    sentences.append(sents)

df_sentences = pd.DataFrame({'date':dates, 
                             'sentences':sentences})
# 정렬: 오름차순
df_sentences = df_sentences.sort_values('date', ignore_index=True)
# 중복값 제거 
df_sentences = df_sentences.drop_duplicates(subset=['date'], ignore_index=True)

print(df_sentences.head(10), df_sentences.tail(10))

df_sentences.to_csv(r'C:\Users\yoons\Desktop\금리예측포르젝트\test\병합할파일\기자회견테스트폴더\conferences_sentences.csv', encoding='utf-8-sig')

Empty DataFrame
Columns: [date, sentences]
Index: [] Empty DataFrame
Columns: [date, sentences]
Index: []


### 디버깅 포함(기자회견 text > 내 사전과 test)

In [7]:
import pandas as pd
import ast  # 문자열 리스트를 파싱하기 위해 필요

# 사전 파일 로드 (파일 경로 확인 필요)
try:
    hawkish_df = pd.read_csv('final_hawkish.csv', encoding='utf-8')
    dovish_df = pd.read_csv('final_dovish.csv', encoding='utf-8')
except FileNotFoundError as e:
    print(f"파일을 찾을 수 없습니다: {e}")
    exit()

# 매파적과 비둘기파적 단어 리스트 (컬럼 이름 확인 필요)
try:
    hawkish_keywords = hawkish_df['Hawkish'].tolist()  # 'Hawkish' 컬럼
    dovish_keywords = dovish_df['Dovish'].tolist()    # 'Dovish' 컬럼
except KeyError as e:
    print(f"컬럼 이름 오류: {e}")
    print("hawkish_df 컬럼:", hawkish_df.columns)
    print("dovish_df 컬럼:", dovish_df.columns)
    exit()

# 기자회견 문서 파일 로드
try:
    conferences_df = pd.read_csv('conferences_sentences.csv', encoding='utf-8')
except FileNotFoundError as e:
    print(f"파일을 찾을 수 없습니다: {e}")
    exit()

# 문장 데이터가 제대로 있는지 확인
if 'sentences' not in conferences_df.columns:
    print("conferences_sentences.csv에 'sentences' 컬럼이 없습니다.")
    print("conferences_df 컬럼:", conferences_df.columns)
    exit()

# 라벨링 함수 (토큰화된 데이터 활용)
def label_sentence(tokens, hawkish_keywords, dovish_keywords):
    # 토큰이 문자열로 저장된 리스트라면 파싱 (예: "[금리, 인상, 예상]" -> 리스트로 변환)
    if isinstance(tokens, str):
        try:
            # 문자열 리스트를 실제 리스트로 변환
            tokens = ast.literal_eval(tokens)
        except (ValueError, SyntaxError):
            # 파싱 실패 시 공백 기준으로 단순 분리
            tokens = tokens.split()
    
    # 토큰이 리스트가 아닌 경우 처리
    if not isinstance(tokens, list):
        return 'neutral'
    
    # 매파적과 비둘기파적 키워드 매칭
    hawkish_count = sum(1 for word in hawkish_keywords if word in tokens)
    dovish_count = sum(1 for word in dovish_keywords if word in tokens)
    
    # 매파적과 비둘기파적 키워드를 비교하여 라벨링
    if hawkish_count > dovish_count:
        return 'hawkish'
    elif dovish_count > hawkish_count:
        return 'dovish'
    else:
        return 'neutral'  # 비슷하게 등장하거나 모두 없는 경우는 중립

# 문서의 각 문장에 라벨링 적용
conferences_df['label'] = conferences_df['sentences'].apply(
    lambda tokens: label_sentence(tokens, hawkish_keywords, dovish_keywords)
)

# 결과 확인
print("라벨링 결과 (상위 5개):")
print(conferences_df.head())

# 결과를 새로운 CSV 파일로 저장
output_file = 'conferences_labeled.csv'
conferences_df.to_csv(output_file, encoding='utf-8-sig', index=False)
print(f"라벨링된 결과가 {output_file}에 저장되었습니다.")

라벨링 결과 (상위 5개):
   Unnamed: 0        date                                          sentences  \
0           0  2009-04-09  ['금융통화위원회는 다음 통화정책방향 결정시까지 한국은행 기준금리를 현 수준(2.0...   
1           1  2009-05-12  ['금융통화위원회는 다음 통화정책방향 결정시까지 한국은행 기준금리를 현 수준(2.0...   
2           2  2009-06-11  ['금융통화위원회는 다음 통화정책방향 결정시까지 한국은행기준금리를 현 수준(2.00...   
3           3  2009-07-09  ['금융통화위원회는 다음 통화정책방향 결정시까지 한국은행기준금리를 현 수준(2.00...   
4           4  2009-08-11  ['금융통화위원회는 다음 통화정책방향 결정시까지 한국은행기준금리를 현 수준(2.00...   

     label  
0  neutral  
1  neutral  
2  neutral  
3  neutral  
4  neutral  
라벨링된 결과가 conferences_labeled.csv에 저장되었습니다.
